In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("reduced.csv", delimiter=',')

# Classify timestamps as datetimes
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index(df['Timestamp'], inplace=True)

In [9]:
# df.drop(['Sin', 'Cos'], axis=1)

def interpolate_linear(df):
    df['sin'] = np.sin(np.deg2rad(df['Wind Direction']))
    df['cos'] = np.cos(np.deg2rad(df['Wind Direction']))

    df_lin = df.copy()
    df_lin.interpolate(inplace=True)#, limit=3)

    df['Wind Direction Linear'] = np.rad2deg(np.arctan2(df_lin['sin'], df_lin['cos'])).apply(full_range)
    df['Wind Speed Linear'] = df_lin['Wind Speed']

    df = df.drop(['sin', 'cos'], axis=1)
    return df

def interpolate_cubic_spline(df):
    df['sin'] = np.sin(np.deg2rad(df['Wind Direction']))
    df['cos'] = np.cos(np.deg2rad(df['Wind Direction']))

    df_spl = df.copy()
    df_spl.interpolate(method='cubic', order=3, inplace=True)#, limit=3) # cubic == spline?
    df['Wind Direction Spline'] = np.rad2deg(np.arctan2(df_spl['sin'], df_spl['cos'])).apply(full_range)
    df['Wind Speed Spline'] = df_spl['Wind Speed']

    df = df.drop(['sin', 'cos'], axis=1)
    return df

def full_range(f):
    if f < 0:
        return f + 360
    else:
        return f

    
    
df = df.resample('3600S').asfreq()[:]

interpolate_linear(df)
interpolate_cubic_spline(df)
df = df.drop(['sin', 'cos'], axis=1)


df['Timestamp'].interpolate(method='time')
# df['2016-04-04']['Wind Direction Spline'].plot()
# df['2016-04-04']['Wind Direction Linear'].plot()



# writer = pd.ExcelWriter('Meteorological Interpolated.xlsx')
# df.to_excel(writer,'ledata')
# writer.save()

,Timestamp,Wind Direction,Wind Speed,Wind Direction Linear,Wind Speed Linear,Wind Direction Spline,Wind Speed Spline
Timestamp,,,,,,,
2016-04-01 00:00:00,2016-04-01 00:00:00,190.500000,4.000000,190.500000,4.000000,190.500000,4.000000
2016-04-01 01:00:00,2016-04-01 01:00:00,194.766667,4.333333,194.766667,4.333333,194.766667,4.333333
2016-04-01 02:00:00,2016-04-01 02:00:00,199.033333,4.666667,199.033333,4.666667,199.033333,4.666667
2016-04-01 03:00:00,2016-04-01 03:00:00,203.300000,5.000000,203.300000,5.000000,203.300000,5.000000
2016-04-01 04:00:00,2016-04-01 04:00:00,202.566667,5.066667,202.566667,5.066667,202.566667,5.066667
2016-04-01 05:00:00,2016-04-01 05:00:00,201.833333,5.133333,201.833333,5.133333,201.833333,5.133333
2016-04-01 06:00:00,2016-04-01 06:00:00,201.100000,5.200000,201.100000,5.200000,201.100000,5.200000
2016-04-01 07:00:00,2016-04-01 07:00:00,202.366667,4.833333,202.366667,4.833333,202.366667,4.833333
2016-04-01 08:00:00,2016-04-01 08:00:00,203.633333,4.466667,203.633333,4.466667,203.633333,4.466667
